# Using btrack with napari


In [1]:
import os
import json

import btrack
import napari

import numpy as np
import pandas as pd

In [2]:
objects = btrack.dataio.import_JSON('objects.json')

In [3]:
# with btrack.dataio.HDF5FileHandler('/Users/arl/Dropbox/Code/py3/arboretum/tracks.h5', 'r') as h:
#     segmentation = h._hdf['segmentation']['images'][:]
#     objects = h.objects

In [4]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file('../models/cell_config.json')

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0,1200), (0,1600), (-1e5,1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimizer
  tracker.optimize()

  # get the tracks in a format for napari visualization
  data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2020/10/02 10:12:39 AM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2020/10/02 10:12:39 AM] btrack (v0.3.12) library imported
[INFO][2020/10/02 10:12:39 AM] Setting max XYZ search radius to: 100
[INFO][2020/10/02 10:12:39 AM] Starting BayesianTracker session
[INFO][2020/10/02 10:12:39 AM] Loading configuration file: ../models/cell_config.json
[INFO][2020/10/02 10:12:39 AM] Loading motion model: b'cell_motion'
[INFO][2020/10/02 10:12:39 AM] Objects are of type: <class 'list'>
[INFO][2020/10/02 10:12:39 AM] Set volume to ((0, 1200), (0, 1600), (-100000.0, 100000.0))
[INFO][2020/10/02 10:12:39 AM] Starting tracking... 
[INFO][2020/10/02 10:12:39 AM] Tracking objects in frames 0 to 99 (of 500)...
[INFO][2020/10/02 10:12:39 AM]  - Timing (Bayesian updates: 1.07ms, Linking: 0.20ms)
[INFO][2020/10/02 10:12:39 AM]  - Probabilities (Link: 1.00000, Lost: 0.48361)
[INFO][2020/10/02 10:12:39 AM]  - Stats (Active: 78, Lost: 191, Conflicts

In [5]:
vertices = data[:, 1:]

In [17]:
# use the tracks PR to visualize the track data
with napari.gui_qt():
    viewer = napari.Viewer()
    #     viewer.add_labels(segmentation)
    viewer.add_points(vertices, size=1, name='points', opacity=0.3)
    viewer.add_tracks(data, name='tracks')
    viewer.layers['tracks'].properties = properties
    viewer.layers['tracks'].graph = graph

### export data for Napari example

In [15]:
np.savetxt('example_track_data.csv', data, delimiter=",", fmt='%.2f')

del properties['track_id'] # this is autogenerated by napari, remove it when saving out
properties_df = pd.DataFrame(properties)
properties_df.to_csv('example_track_properties.csv')

with open('example_track_graph.txt', 'w') as file:
    print(graph, file=file)